# OMMX Message

OMMX defines two data formats, [OMMX Message](https://github.com/Jij-Inc/ommx/blob/main/MESSAGE.md) and [OMMX Artifact](https://github.com/Jij-Inc/ommx/blob/main/ARTIFACT.md). This notebook is a tutorial for OMMX Message.

## Create Instance

`Instance` is a data structure corresponding to a mathematical programming problem. For example, let us consider the Knapsack problem

$$
\begin{align*}
& \text{Maximize} & \sum_{i=1}^n p_i x_i & \\
&\text{Subject to} & \sum_{i=1}^n w_i x_i \leq W & ,\space x_i \in \{0, 1\} \quad (i = 1, \ldots, n)
\end{align*}
$$

where $p_i$ is the profit of item $i$, $w_i$ is the weight of item $i$, and $W$ is the capacity of the knapsack. The binary variables $x_i$ to be determined by solving this problem are called decision variables.

The `Instance` of the Knapsack problem is defined by the following:

In [1]:
from ommx.v1 import Instance, DecisionVariable

p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
W = 47

x = [DecisionVariable.binary(i) for i in range(6)]

objective = sum(p[i] * x[i] for i in range(6))
constraint = sum(w[i] * x[i] for i in range(6)) <= W

instance = Instance.from_components(
    decision_variables=x,
    objective=objective,
    constraints=[constraint],
    sense=Instance.MAXIMIZE,
)


In OMMX, all decision variables have their ID, the `i` passed to `DecisionVariable.binary` function in the above example. This must be unique in the `Instance`. The `DecisionVariable` class represents a decision variable, i.e. its ID and kind:

In [2]:
x[1]

DecisionVariable(raw=id: 1
kind: KIND_BINARY
)

Since the value of decision variables are not determined until the problem is solved, we rather treat their function e.g. $f(x_1, x_2) = 2 x_1 + 3x_2$. Using the ID of decision variables, this linear function is represented as a pair of ID and its coefficient:

In [3]:
2 * x[1] + 3 * x[2]

Linear(raw=terms {
  id: 1
  coefficient: 2
}
terms {
  id: 2
  coefficient: 3
}
)

The `DecisionVariable` object can be used as a placeholder, and mathematical operator `+` and `*` are overloaded to create a linear function. It can be used for the objective function. For constraint, we can take `==` of the linear function object for equality constraint, and `<=` for inequality constraint.

In [4]:
3 * x[1] + 4 * x[3] <= 10

Constraint(raw=id: 1
equality: EQUALITY_LESS_THAN_OR_EQUAL_TO_ZERO
function {
  linear {
    terms {
      id: 1
      coefficient: 3
    }
    terms {
      id: 3
      coefficient: 4
    }
    constant: -10
  }
}
)

The constraints also have unique ID. This is automatically generated by creating a `Constraint` object like above process. You can also create a `Constraint` object with a specific ID by passing it as an argument. See the [API reference](https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/constraint_pb2/index.html#ommx.v1.constraint_pb2.Constraint) for details.

In the above Knapsack example, the profit $p_i$ and weights $w_i$ are embedded as the coefficient of linear functions and composed into `instance` object. `instance` object is a collection of decision variables, constraints, and objective function. OMMX Message defines how to serialize and deserialize this object:

In [5]:
byte_array = instance.to_bytes()

This `byte_array` is a binary representation of the `Instance` object based on the [OMMX Message schema](https://jij-inc.github.io/ommx/protobuf.html) defined by the [Protocol Buffers](https://protobuf.dev/). The `Instance` object can be deserialized from this binary representation by calling `Instance.from_bytes` method.

In [6]:
instance = Instance.from_bytes(byte_array)

The main advantage of Protocol Buffers is that it is language and platform independent. The `Instance` object can be serialized in Python and deserialized in other languages such as C++ or Rust and vice versa. This is useful when you want to create a problem in Python on your laptop, and solve it in C++ or Rust on a server. See [MESSAGE.md](https://github.com/Jij-Inc/ommx/blob/main/MESSAGE.md) for more details about entire design.

Note that OMMX is not designed as a modeler library, and modeler API is very limited. The above example is just a demonstration of what data is stored in `Instance` object for better understanding of serialized data.

## Solve using Python-MIP

OMMX project manages an adapter package `ommx-python-mip-adapter`, which provides a bridge between OMMX and Python-MIP. Python-MIP is a Python library for mathematical programming, which provides a high-level modeling API. The adapter package converts `Instance` object to Python-MIP model, and vice versa.

Let's solve the above Knapsack instance using Python-MIP:

In [7]:
import ommx_python_mip_adapter as adapter

result = adapter.solve(instance)

Cbc0038I Initial state - 1 integers unsatisfied sum - 0.457143
Cbc0038I Solution found of 28
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 6 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of 30.3171
Cbc0038I Reduced cost fixing fixed 1 variables on major pass 2
Cbc0038I Pass   1: suminf.    0.07474 (1) obj. 30.3171 iterations 1
Cbc0038I Pass   2: suminf.    0.45714 (1) obj. 42.1714 iterations 1
Cbc0038I Pass   3: suminf.    0.07474 (1) obj. 30.3171 iterations 1
Cbc0038I Pass   4: suminf.    0.45714 (1) obj. 42.1714 iterations 1
Cbc0038I Pass   5: suminf.    0.34461 (1) obj. 30.3171 iterations 2
Cbc0038I Solution found of 41
Cbc0038I Before mini branch and bound, 4 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 6 columns, reduced to 1 rows 2 columns
Cbc0038I Mini branch and bound did not improve solution (0.

`result` is object of type `ommx.v1.Result` which contains `solution` field if solved. This may contain `infeasible` field if the problem is infeasible, i.e. the solver prove that all solution of the problem is infeasible, or `unbounded` field if the problem is unbounded.

In [8]:
assert result.HasField("solution")  # Above KnapSack problem has a solution
solution = result.solution

The main part of the solution is the values of the decision variables $x_i$. They are stored in `state` field:

In [9]:
solution.state.entries

{3: 1.0, 5: 0.0, 2: 0.0, 1: 0.0, 0: 1.0, 4: 0.0}

This says $x[0] = x[3] = 1$ and others are $0$. Then the profit is $p[0] + p[3] = 10 + 31 = 41$.

In [10]:
solution.objective

41.0

Substituting $x$, the constraint becomes $w[0] + w[3] = 11 + 35 = 46 \leq 47 = W$. Since the inequality constraints are normalized into $f(x) \leq 0$ form, the value of the constraint is $46 - 47 = -1$. This is stored in `evaluated_value` field.

In [11]:
assert len(solution.evaluated_constraints) == 1  # There is only one constraint
constraint = solution.evaluated_constraints[0]
constraint.evaluated_value

-1.0